# Data Collection - NCAA Basketball 2025-26

This notebook collects team statistics from various sources for the 2025-26 season.

**Primary Data Source:** Barttorvik (free efficiency ratings)  
**Optional Sources:** ESPN, Haslametrics, CBBpy (for future features)

**Data we collect:**
- Adjusted Offensive Efficiency (AdjO)
- Adjusted Defensive Efficiency (AdjD)
- Win/Loss records and win percentage
- Power ratings (net efficiency)

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from io import StringIO
import ssl
import urllib.request
from urllib.error import URLError, HTTPError
import certifi
import time
import warnings
warnings.filterwarnings('ignore')

from src import config

print("Libraries loaded!")

Libraries loaded!


---

## Part 1: Barttorvik Data (Primary Source)

Barttorvik provides free, reliable efficiency ratings used as our main data source.

### 1.1 Target Teams

In [2]:
# Teams from our prediction template
TARGET_TEAMS = [
    'Baylor', 'Boston College', 'California', 'Clemson', 'Duke',
    'Florida State', 'Georgia Tech', 'Louisville', 'Miami', 'Michigan',
    'NC State', 'North Carolina', 'Notre Dame', 'Ohio State', 'Pitt',
    'SMU', 'Stanford', 'Syracuse', 'Virginia', 'Virginia Tech', 'Wake Forest'
]

# Mapping from our schedule names to Barttorvik names
SCHEDULE_TO_BARTTORVIK = {
    'Florida State': 'Florida St.',
    'Miami': 'Miami FL',
    'NC State': 'N.C. State',
    'Ohio State': 'Ohio St.',
    'Pitt': 'Pittsburgh',
}

print(f"Need data for {len(TARGET_TEAMS)} teams")

Need data for 21 teams


### 1.2 Scrape Barttorvik CSV

In [3]:
def scrape_barttorvik_csv(year=2026, max_retries=3, retry_delay=1.0):
    """Fetch team ratings from Barttorvik CSV endpoint with retry logic"""
    url = f"https://barttorvik.com/{year}_team_results.csv"
    
    try:
        print(f"Fetching Barttorvik CSV for {year}...")
        
        # Try with requests first
        try:
            response = requests.get(url, timeout=15)
            response.raise_for_status()
            df = pd.read_csv(StringIO(response.text), index_col=False)  # Fix: prevent column shift
            print(f"✓ Found {len(df)} teams")
            return df
        except:
            pass
        
        # Try with secure SSL using certifi
        ssl_context = ssl.create_default_context(cafile=certifi.where())
        
        for attempt in range(max_retries):
            try:
                with urllib.request.urlopen(url, context=ssl_context, timeout=30) as response:
                    data = response.read().decode('utf-8')
                    df = pd.read_csv(StringIO(data), index_col=False)  # Fix: prevent column shift
                    print(f"✓ Found {len(df)} teams")
                    return df
            except (URLError, HTTPError, ssl.SSLError) as e:
                if attempt < max_retries - 1:
                    wait_time = retry_delay * (2 ** attempt)
                    print(f"   Attempt {attempt + 1}/{max_retries} failed, retrying...")
                    time.sleep(wait_time)
                else:
                    # ⚠️ WARNING: Fallback without SSL verification (INSECURE!)
                    print(f"   ⚠️ All {max_retries} attempts failed with SSL verification")
                    print(f"   ⚠️ WARNING: Attempting without SSL verification (INSECURE!)")
                    ssl_context_unverified = ssl.create_default_context()
                    ssl_context_unverified.check_hostname = False
                    ssl_context_unverified.verify_mode = ssl.CERT_NONE
                    with urllib.request.urlopen(url, context=ssl_context_unverified, timeout=30) as response:
                        data = response.read().decode('utf-8')
                        df = pd.read_csv(StringIO(data), index_col=False)
                        print(f"✓ Found {len(df)} teams (fallback)")
                        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

# Fetch the data
barttorvik_df = scrape_barttorvik_csv(2026)

if barttorvik_df is not None:
    print(f"\nColumns: {len(barttorvik_df.columns)}")
    display(barttorvik_df.head(10))

Fetching Barttorvik CSV for 2026...
✓ Found 365 teams

Columns: 45


,rank,team,conf,record,adjoe,oe Rank,adjde,de Rank,barthag,rank.1,...,ConPA,ConPoss,ConOE,ConDE,ConSOSRemain,Conf Win%,WAB,WAB Rk,Fun Rk,adjt
0,1,Michigan,B10,20-1,127.137693,5,90.282171,1,0.980863,1,...,792.0,787.5125,1.194902,1.005698,0.852529,0.909091,7.825482,4,63,72.692022
1,2,Arizona,B12,22-0,126.113533,8,91.891913,3,0.974433,2,...,647.0,659.7500,1.205002,0.980674,0.886786,1.000000,8.257857,1,7,71.124624
2,3,Houston,B12,20-2,127.772711,4,93.247086,6,0.973979,3,...,570.0,580.4000,1.228463,0.982081,0.823219,0.888889,5.804556,7,131,63.299303
3,4,Illinois,B10,20-3,131.620625,1,97.173125,20,0.970386,4,...,818.0,770.9250,1.254337,1.061063,0.844000,0.916667,6.522936,5,77,65.650642
4,5,Iowa St.,B12,20-2,126.579074,6,94.125701,11,0.967914,5,...,609.0,608.7500,1.197536,1.000411,0.863580,0.777778,5.528334,8,79,67.821284
5,6,Duke,ACC,21-1,125.087326,11,93.197800,5,0.967211,6,...,648.0,661.9125,1.216173,0.978981,0.850910,1.000000,7.934851,2,11,66.427835
6,7,Connecticut,BE,22-1,122.714874,27,91.634527,2,0.966387,7,...,788.0,814.5625,1.179774,0.967390,0.804340,1.000000,7.892746,3,16,64.844679
7,8,Florida,SEC,16-6,124.841492,14,93.460987,8,0.965419,8,...,655.0,643.1875,1.231367,1.018366,0.827264,0.777778,3.360372,21,334,69.658640
8,9,Purdue,B10,18-4,130.129508,2,98.323964,27,0.961691,9,...,781.0,720.0500,1.230470,1.084647,0.879456,0.727273,5.465013,9,139,64.977067
9,10,Vanderbilt,SEC,19-3,125.011357,13,95.315218,15,0.957669,10,...,684.0,633.9000,1.148446,1.079035,0.861150,0.666667,4.900139,12,127,70.023814


In [4]:
# Data Validation - ensure scraped data is correct
if barttorvik_df is not None:
    print("Validating scraped data...")
    
    # Check required columns exist
    required_cols = ['team', 'adjoe', 'adjde', 'record']
    missing_cols = [col for col in required_cols if col not in barttorvik_df.columns]
    if missing_cols:
        print(f"❌ ERROR: Missing columns: {missing_cols}")
    else:
        print(f"✓ All required columns present")
    
    # Check data types and ranges
    assert barttorvik_df['team'].dtype == object, "team should be string type"
    assert len(barttorvik_df) >= 350, f"Expected ~365 teams, got {len(barttorvik_df)}"
    
    # Validate efficiency ranges (typical: 90-140 for adjoe, 85-115 for adjde)
    adjoe_range = (barttorvik_df['adjoe'].min(), barttorvik_df['adjoe'].max())
    adjde_range = (barttorvik_df['adjde'].min(), barttorvik_df['adjde'].max())
    
    assert 80 < adjoe_range[0] < 100, f"adjoe min seems wrong: {adjoe_range[0]}"
    assert 130 < adjoe_range[1] < 150, f"adjoe max seems wrong: {adjoe_range[1]}"
    assert 80 < adjde_range[0] < 100, f"adjde min seems wrong: {adjde_range[0]}"
    assert 110 < adjde_range[1] < 130, f"adjde max seems wrong: {adjde_range[1]}"
    
    print(f"✓ Efficiency ranges valid: adjoe {adjoe_range[0]:.1f}-{adjoe_range[1]:.1f}, adjde {adjde_range[0]:.1f}-{adjde_range[1]:.1f}")
    
    # Check for data corruption (conference codes in team column)
    first_teams = barttorvik_df['team'].head(5).tolist()
    conf_codes = ['B12', 'B10', 'ACC', 'SEC', 'BE', 'WCC', 'A10', 'MWC', 'P12']
    corrupted_teams = [t for t in first_teams if t in conf_codes]
    if corrupted_teams:
        print(f"❌ ERROR: Team column contains conference codes: {corrupted_teams}")
        print(f"   This indicates data corruption - check index_col parameter!")
        raise ValueError("Data corruption detected!")
    else:
        print(f"✓ Team names valid (no conference codes detected)")
    
    # Check index starts at 0
    assert barttorvik_df.index[0] == 0, f"Index should start at 0, got {barttorvik_df.index[0]}"
    print(f"✓ Index starts at 0 (no row skipping)")
    
    print("\n✅ All validation checks passed!")

Validating scraped data...
✓ All required columns present
✓ Efficiency ranges valid: adjoe 88.1-131.6, adjde 90.3-128.3
✓ Team names valid (no conference codes detected)
✓ Index starts at 0 (no row skipping)

✅ All validation checks passed!


### 1.3 Process and Filter Data

In [5]:
# Name mappings
BARTTORVIK_TO_SCHEDULE = {v: k for k, v in SCHEDULE_TO_BARTTORVIK.items()}

SCHEDULE_TEAMS = [
    'Baylor', 'Boston College', 'California', 'Clemson', 'Duke',
    'Florida State', 'Georgia Tech', 'Louisville', 'Miami', 'Michigan',
    'NC State', 'North Carolina', 'Notre Dame', 'Ohio State', 'Pitt',
    'SMU', 'Stanford', 'Syracuse', 'Virginia', 'Virginia Tech', 'Wake Forest'
]

def process_barttorvik_csv(df):
    """Process and filter to target teams"""
    if df is None:
        return None
    
    # Find team column
    team_col = next((col for col in df.columns if col.lower() == 'team'), None)
    if not team_col:
        print("Error: Could not find team column")
        return None
    
    result = df.copy()
    result['team_clean'] = result[team_col].apply(
        lambda x: BARTTORVIK_TO_SCHEDULE.get(str(x).strip(), str(x).strip())
    )
    
    filtered = result[result['team_clean'].isin(SCHEDULE_TEAMS)].copy()
    print(f"Found {len(filtered)}/{len(SCHEDULE_TEAMS)} target teams")
    
    missing = set(SCHEDULE_TEAMS) - set(filtered['team_clean'].tolist())
    if missing:
        print(f"Missing teams: {missing}")
    
    return filtered

our_teams = process_barttorvik_csv(barttorvik_df)
if our_teams is not None:
    display(our_teams[['team_clean', 'adjoe', 'adjde', 'record', 'barthag']].sort_values('barthag', ascending=False))

Found 21/21 target teams


,team_clean,adjoe,adjde,record,barthag
0,Michigan,127.137693,90.282171,20-1,0.980863
5,Duke,125.087326,93.197800,21-1,0.967211
14,Virginia,124.046537,97.593385,19-3,0.940377
18,Louisville,123.921963,98.128864,16-6,0.936062
23,NC State,123.154772,99.774683,17-6,0.918420
24,Clemson,116.310019,94.255698,19-4,0.918178
27,North Carolina,123.877292,101.388413,18-4,0.909193
33,SMU,124.539107,104.088053,15-7,0.887245
38,Ohio State,122.892075,104.099530,14-7,0.870854
43,Miami,119.161662,102.131534,17-5,0.854895


def find_col(df, options):
    """Find column by name options"""
    for opt in options:
        matches = [c for c in df.columns if opt.lower() in c.lower()]
        if matches:
            return matches[0]
    return None

if our_teams is not None and len(our_teams) >= 18:
    print("✓ Using scraped Barttorvik data")
    
    adj_o_col = find_col(our_teams, ['adjoe', 'adj_o'])
    adj_d_col = find_col(our_teams, ['adjde', 'adj_d'])
    
    # Parse wins/losses from record column (format: '16-2')
    our_teams['wins'] = our_teams['record'].str.split('-').str[0].astype(float)
    our_teams['losses'] = our_teams['record'].str.split('-').str[1].astype(float)
    
    model_df = pd.DataFrame({
        'team': our_teams['team_clean'].values,
        'off_efficiency': our_teams[adj_o_col].values,
        'def_efficiency': our_teams[adj_d_col].values,
        'wins': our_teams['wins'].values,
        'losses': our_teams['losses'].values
    })
    
    # Calculate derived features
    # Note: ppg, opp_ppg, pace are NOT used by the model (see config.BASELINE_FEATURES)
    # They're calculated for reference only. The model uses efficiency values directly.
    model_df['power_rating'] = model_df['off_efficiency'] - model_df['def_efficiency']
    model_df['win_pct'] = model_df['wins'] / (model_df['wins'] + model_df['losses'])
    
    print(f"\n✓ Created model data for {len(model_df)} teams")
    display(model_df.sort_values('power_rating', ascending=False))
else:
    print("⚠️  No Barttorvik data available")

In [6]:
def find_col(df, options):
    """Find column by name options"""
    for opt in options:
        matches = [c for c in df.columns if opt.lower() in c.lower()]
        if matches:
            return matches[0]
    return None

if our_teams is not None and len(our_teams) >= 18:
    print("✓ Using scraped Barttorvik data")
    
    adj_o_col = find_col(our_teams, ['adjoe', 'adj_o'])
    adj_d_col = find_col(our_teams, ['adjde', 'adj_d'])
    
    # Parse wins/losses from record column (format: '16-2')
    our_teams['wins'] = our_teams['record'].str.split('-').str[0].astype(float)
    our_teams['losses'] = our_teams['record'].str.split('-').str[1].astype(float)
    
    model_df = pd.DataFrame({
        'team': our_teams['team_clean'].values,
        'off_efficiency': our_teams[adj_o_col].values,
        'def_efficiency': our_teams[adj_d_col].values,
        'wins': our_teams['wins'].values,
        'losses': our_teams['losses'].values
    })
    
    model_df['ppg'] = model_df['off_efficiency'] * 0.70
    model_df['opp_ppg'] = model_df['def_efficiency'] * 0.70
    model_df['pace'] = 70.0
    model_df['power_rating'] = model_df['off_efficiency'] - model_df['def_efficiency']
    model_df['win_pct'] = model_df['wins'] / (model_df['wins'] + model_df['losses'])
    
    print(f"\n✓ Created model data for {len(model_df)} teams")
    display(model_df.sort_values('power_rating', ascending=False))
else:
    print("⚠️  No Barttorvik data available")

✓ Using scraped Barttorvik data

✓ Created model data for 21 teams


,team,off_efficiency,def_efficiency,wins,losses,ppg,opp_ppg,pace,power_rating,win_pct
0,Michigan,127.137693,90.282171,20.0,1.0,88.996385,63.197520,70.0,36.855523,0.952381
1,Duke,125.087326,93.197800,21.0,1.0,87.561128,65.238460,70.0,31.889526,0.954545
2,Virginia,124.046537,97.593385,19.0,3.0,86.832576,68.315369,70.0,26.453152,0.863636
3,Louisville,123.921963,98.128864,16.0,6.0,86.745374,68.690204,70.0,25.793100,0.727273
4,NC State,123.154772,99.774683,17.0,6.0,86.208340,69.842278,70.0,23.380089,0.739130
6,North Carolina,123.877292,101.388413,18.0,4.0,86.714104,70.971889,70.0,22.488879,0.818182
5,Clemson,116.310019,94.255698,19.0,4.0,81.417013,65.978989,70.0,22.054321,0.826087
7,SMU,124.539107,104.088053,15.0,7.0,87.177375,72.861637,70.0,20.451055,0.681818
8,Ohio State,122.892075,104.099530,14.0,7.0,86.024453,72.869671,70.0,18.792546,0.666667
9,Miami,119.161662,102.131534,17.0,5.0,83.413163,71.492074,70.0,17.030128,0.772727


### 1.5 Save to File

In [7]:
output_path = config.PROCESSED_DATA_DIR / 'team_stats_2025_26.csv'
model_df.to_csv(output_path, index=False)
print(f"✓ Saved to {output_path}")
print(f"  {len(model_df)} teams")
print(f"  Power rating range: {model_df['power_rating'].min():.1f} to {model_df['power_rating'].max():.1f}")

✓ Saved to /Users/calebhan/Documents/Coding/Personal/triangle-sports-analytics-26/notebooks/../data/processed/team_stats_2025_26.csv
  21 teams
  Power rating range: 2.4 to 36.9


HOME_COURT_ADVANTAGE = 3.5

def predict_spread(home_team, away_team, team_stats):
    """
    SIMPLE sanity check spread estimate (NOT the actual model!).
    
    This uses a simplified power rating formula for quick validation.
    The actual model in 02_modeling.ipynb uses Ridge + LightGBM ensemble
    with Elo ratings and multiple features.
    """
    stats = team_stats.set_index('team')
    home_net = stats.loc[home_team, 'power_rating']
    away_net = stats.loc[away_team, 'power_rating']
    # Simple formula: efficiency differential / 2 + home court advantage
    spread = (home_net - away_net) / 2 + HOME_COURT_ADVANTAGE
    return spread

test_matchups = [
    ('North Carolina', 'Duke'),
    ('Duke', 'North Carolina'),
    ('Virginia', 'Duke'),
]

print("Simple Spread Estimates (sanity check - not actual model):")
print("=" * 50)
for home, away in test_matchups:
    spread = predict_spread(home, away, model_df)
    if spread > 0:
        print(f"{away:15} @ {home:15} → {home} by {spread:.1f}")
    else:
        print(f"{away:15} @ {home:15} → {away} by {-spread:.1f}")

In [8]:
HOME_COURT_ADVANTAGE = 3.5

def predict_spread(home_team, away_team, team_stats):
    """Simple spread prediction using net efficiency"""
    stats = team_stats.set_index('team')
    home_net = stats.loc[home_team, 'power_rating']
    away_net = stats.loc[away_team, 'power_rating']
    spread = (home_net - away_net) / 2 + HOME_COURT_ADVANTAGE
    return spread

test_matchups = [
    ('North Carolina', 'Duke'),
    ('Duke', 'North Carolina'),
    ('Virginia', 'Duke'),
]

print("Expected Spreads (sanity check):")
print("=" * 50)
for home, away in test_matchups:
    spread = predict_spread(home, away, model_df)
    if spread > 0:
        print(f"{away:15} @ {home:15} → {home} by {spread:.1f}")
    else:
        print(f"{away:15} @ {home:15} → {away} by {-spread:.1f}")

Expected Spreads (sanity check):
Duke            @ North Carolina  → Duke by 1.2
North Carolina  @ Duke            → Duke by 8.2
Duke            @ Virginia        → Virginia by 0.8


---

## Part 2: Optional Data Sources (Future Features)

These sources can provide additional signals for future model improvements.

### 2.1 ESPN BPI (Optional)

ESPN's Basketball Power Index provides alternative team ratings.

**Status:** Not currently used (Barttorvik is sufficient)  
**Potential use:** Test as additional feature if model plateaus

In [9]:
# Uncomment to explore ESPN data
# from src.data_sources import espn
# 
# standings = espn.fetch_standings(year=2026)
# print(f"Found {len(standings)} teams with standings")
# display(standings.head(10))

### 2.2 Haslametrics (Optional)

Haslametrics provides momentum metrics and consistency scores.

**Status:** Not currently used  
**Potential use:** Momentum features for blowout prediction (+2-5% MAE improvement expected)

In [10]:
# Uncomment to explore Haslametrics data
# from src.data_sources import haslametrics
# 
# ratings = haslametrics.fetch_team_ratings(2026)
# momentum = haslametrics.fetch_momentum_metrics(2026)
# 
# print(f"Ratings: {len(ratings)} teams")
# print(f"Momentum: {len(momentum)} teams")
# 
# if len(momentum) > 0:
#     print("\nTop teams by momentum:")
#     display(momentum.head(10))

### 2.3 CBBpy (Optional)

CBBpy provides play-by-play, box scores, and player-level stats from NCAA.com.

**Status:** Not currently used (high effort, moderate value)  
**Potential use:** Pace variance, bench depth, run analysis

**Note:** Requires `pip install cbbpy` and one-time patch:
```bash
python scripts/patch_cbbpy_venv.py
```

In [11]:
# Uncomment to explore CBBpy data
# from src.data_sources import cbbpy_enhanced
# 
# if cbbpy_enhanced.CBBPY_AVAILABLE:
#     games = cbbpy_enhanced.fetch_games_team('Duke', season=2026)
#     print(f"Found {len(games)} Duke games")
#     display(games.head(10))
# else:
#     print("⚠️  CBBpy not installed")
#     print("Install with: pip install cbbpy")

---

## Summary

✅ **Barttorvik data collected** - 21 teams with efficiency ratings  
✅ **Saved to:** `data/processed/team_stats_2025_26.csv`  
📊 **Next step:** Run `02_modeling.ipynb` to train the prediction model

**Optional data sources** (ESPN, Haslametrics, CBBpy) are available for future feature engineering but not currently needed.